<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
print('start non-rigid fitting')
	nonrigid_optimizer = torch.optim.Adam([id_tensor, tex_tensor,
										exp_tensor, rot_tensor,
										gamma_tensor, trans_tensor], lr=0.01)
	for i in range(200):
		nonrigid_optimizer.zero_grad()
		coeff = torch.cat([id_tensor, exp_tensor,
						tex_tensor, rot_tensor,
						gamma_tensor, trans_tensor], dim=1)
		rendered_img, pred_lms, face_texture, _ = model(coeff)
		mask = rendered_img[:, :, :, 3].detach()
		photo_loss_val = photo_loss(rendered_img[:, :, :, :3], img_tensor, mask>0)
		lm_loss_val = lm_loss(pred_lms, lms, img_size=TAR_SIZE)
		reg_loss_val = reg_loss(id_tensor, exp_tensor, tex_tensor)
		tex_loss_val = reflectance_loss(face_texture, skinmask)
		loss = photo_loss_val*NRF_PHOTO_LOSS_W + \
						lm_loss_val*NRF_LM_LOSS_W + \
						reg_loss_val*NRF_REG_W + \
						tex_loss_val*NRF_TEX_LOSS_W
		loss.backward()
		nonrigid_optimizer.step()